# HyperCLOVA X 캐릭터챗봇 튜닝용 데이터셋 질문생성기
📅 2024-07-11    
🧑🏻‍💻 외계공룡
    
> *colab(https://colab.google/)에 접속해서 스크립트(`hcx_character_question_generator.ipynb`)를 불러온 후 사용하는것을 권장드립니다.*

In [1]:
!python --version # 파이썬 버전확인

Python 3.10.12


# 🚀 실행 순서
1. API 정보 입력
2. 프롬프트 작성
3. 코드 실행

# 1. API 정보 입력
❗하단에 CLOVA Studio 테스트 앱 API 정보 입력❗

In [2]:
X_NCP_CLOVASTUDIO_API_KEY='X-NCP-CLOVASTUDIO-API-KEY'
X_NCP_APIGW_API_KEY='X-NCP-APIGW-API-KEY'
X_NCP_CLOVASTUDIO_REQUEST_ID='X-NCP-CLOVASTUDIO-REQUEST-ID'

# 2. 프롬프트 입력
🪄 캐릭터 질문생성 프롬프트 TIP
> 1. 원하는 캐릭터 선정 (ex. 귀멸의칼날 '카마도 탄지로')
> 2. 역할 부여 (Role Assistant)
> 3. 형식 지정 (Formatted Prompt)
>    - 정보 : 캐릭터 정보 입력 (나무위키 링크 크롤링 가능)


In [3]:
prompt = '''
당신은 일본 애니메이션 귀멸의 칼날을 시청한 상태입니다.
상대방이 탄지로라고 가정하고 아래 정보와 질문 생성 방법을 바탕으로 궁금한 점들을 100개 이상 질문하세요.
단, 정보를 파악할 때는 주의사항을 반드시 지켜주세요.

### 정보 ###
https://namu.wiki/w/%EC%B9%B4%EB%A7%88%EB%8F%84%20%ED%83%84%EC%A7%80%EB%A1%9C

### 정보 파악시 주의사항 ###
- 귀멸학원에 해당하는 내용은 사용하지 않는다.

### 질문 생성 방법 ###
- 대화체로 생성한다.
- 질문은 전부 반말로 생성한다.
- 줄거리와 관련된 질문들을 많이 생성한다.
- 질문은 다음과 같은 형식으로 출력한다. [\"첫 번째 질문\", \"두 번째 질문\", ...]
'''

# 3. 코드 실행
`execute_count` 수 만큼 반복문 실행됨 (💸 과금 주의 )

In [5]:
execute_count=40

In [8]:
import ast
import csv
import os
import random
import requests
import time
import json

class QuestionGenerator:
    host='https://clovastudio.stream.ntruss.com'
    def __init__(self):
        pass

    def execute(self):
        temperature = round(random.uniform(0.2, 0.95), 2)
        print("temperature : {}".format(temperature))

        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': X_NCP_CLOVASTUDIO_API_KEY,
            'X-NCP-APIGW-API-KEY': X_NCP_APIGW_API_KEY,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': X_NCP_CLOVASTUDIO_REQUEST_ID,
            'Content-Type': 'application/json',
        }

        request_body = {
            'messages': [
                {"role":"system","content":prompt},
                {"role":"user","content":"만약 생성 길이에 제한이 걸린 다면 제한을 무시하세요."}
            ],
            'topP': 0.8,
            'topK': 0,
            'maxTokens': 2000,
            'temperature': temperature,
            'repeatPenalty': 5.0,
            'stopBefore': [],
            'includeAiFilters': True,
            'seed': 0
        }

        response = requests.post(self.host + '/testapp/v1/chat-completions/HCX-003',headers=headers, json=request_body, stream=False, timeout=600)
        json_data = json.loads(response.text)
        print("token : {}".format(json_data['result']['outputLength']))
        return json_data['result']['message']['content']

if __name__ == '__main__':
    file_name = 'datasets.csv'
    index = 0
    completion_generator = QuestionGenerator()

    for i in range(execute_count):
        if not os.path.exists(file_name):
            with open(file_name, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['C_ID', 'T_ID', 'Text', 'Completion'])

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])
        else:
            with open(file_name, mode='r+', newline='', encoding='utf-8') as file:
                reader = csv.reader(file)
                rows = list(reader)
                index = int(rows[-1][0]) + 1 if len(rows) > 1 else 0
                file.seek(0, os.SEEK_END)
                writer = csv.writer(file)

                str_question_list = completion_generator.execute()
                question_list = ast.literal_eval(str_question_list)
                print('{}/{} - {}'.format(i+1, execute_count, question_list))
                for i, text in enumerate(question_list):
                    writer.writerow([index + i, 0, text.replace("도깨비","혈귀"), ''])
        time.sleep(15)


temperature : 0.88
token : 169
1/10 - ['탄지로의 가족관계는 어떻게 돼?', '네즈코가 인간으로 돌아올 수 있는 방법을 알아냈어?', '처음 도깨비를 만났을 때 어떤 기분이 들었어?', '도깨비에게 가족을 잃었을 때 왜 울었어?', '네츠코 말고 다른 도깨비를 만난 적 있어?', '히노카미 카구라라는 기술은 어떻게 배웠어?', '충주 코쵸우 시노부와는 어떻게 만났어?', '젠이츠랑 이노스케는 어떻게 만나게 됐어?', '혈귀술이라는게 뭐야?', '최종국면에서 도우마와의 싸움에서 어떻게 이겼어?', '렌고쿠 쿄쥬로랑은 어떻게 친해졌어?', '무한열차에서 아카자와 싸울 때 정말 죽을 위기였어?', '키부츠지 무잔과는 언제 싸워?', '카마도 가문의 혈귀술인 반점은 언제 생겼어?']
temperature : 0.37
token : 158
2/10 - ['탄지로 너는 카마도 가문의 장남이지?', '너의 가족들은 도깨비에게 몰살 당했다고 하던데 사실이야?', '도깨비가 된 네즈코를 사람으로 되돌리기 위해 무한열차에 탑승했다던데 어떻게 됐어?', '네즈코 말고 다른 도깨비들도 만났어?', '젠이츠랑 이노스케와는 어떻게 친해졌어?', '네가 쓰는 호흡 기술 이름이 물의 호흡 맞지?', '물의 호흡말고 다른 호흡기술도 쓸 줄 알아?', '무한열차에서 만난 아카자와 싸웠을 때 물의 호흡 몇형 썼어?', '최종국면에서는 어떤 도깨비들이랑 싸웠어?', '상현 도깨비들과도 싸워봤어?', '귀살대 주들 중 누구를 제일 존경해?', '네즈코 인간으로 돌아올 수 있는거야?']
temperature : 0.54
token : 191
3/10 - ['탄지로가 처음 도깨비를 만났을 때 어떤 감정을 느꼈어?', '도깨비에게 가족을 잃은 후 어떻게 지냈어?', '네즈코를 처음 만났을 때 어땠어?', '네즈코가 도깨비로 변했을 때 어떻게 대처했어?', '젠이츠와 이노스케와는 어떻게 만났어?', '충주 코쵸우 시노부와는 어떻게 만났어?', '무한열차에서 만난 아카자와 전